In [2]:
import os

import torch
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


In [3]:
print( os.getcwd() )
os.chdir( "/var/model/" )
print( os.getcwd() )

/
/var/model


In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
) #

In [5]:
tokenizer = AutoTokenizer.from_pretrained( "." )

In [6]:
model = AutoModelForCausalLM.from_pretrained( ".", quantization_config=bnb_config, device_map="auto", low_cpu_mem_usage=True )

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training( model )  # prepares the whole model for kbit training
type( model )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

transformers.models.llama.modeling_llama.LlamaForCausalLM

In [7]:
lora_config = LoraConfig(
    r=16, 
    lora_alpha=32, 
    target_modules=[ "q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj" ], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

In [8]:
model = get_peft_model( model, lora_config )
model.print_trainable_parameters()

trainable params: 62,586,880 || all params: 13,078,615,040 || trainable%: 0.4785436363757366
